In [ ]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Lambda, Input
import gym

In [ ]:
"""Ivmech PID Controller is simple implementation of a Proportional-Integral-Derivative (PID) Controller in the Python Programming Language.
More information about PID Controller: http://en.wikipedia.org/wiki/PID_controller
"""

class PID:
    """PID Controller
    """

    def __init__(self, P=0.2, I=0.0, D=0.0, current_time=None):

        self.Kp = P
        self.Ki = I
        self.Kd = D

        self.sample_time = 0.00
        self.current_time = current_time if current_time is not None else time.time()
        self.last_time = self.current_time

        self.clear()

    def clear(self):
        """Clears PID computations and coefficients"""
        self.SetPoint = np.array([1, 0])

        self.PTerm = np.zeros(2)
        self.ITerm = np.zeros(2)
        self.DTerm = np.zeros(2)
        self.last_error = np.zeros(2)

        # Windup Guard
        self.int_error = np.zeros(2)
        self.windup_guard = np.array([20.0])

        self.output = np.zeros(2)

    def update(self, feedback_value, current_time=None):
        """Calculates PID value for given reference feedback

        .. math::
            u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}

        .. figure:: images/pid_1.png
           :align:   center

           Test PID with Kp=1.2, Ki=1, Kd=0.001 (test_pid.py)

        """
        error = self.SetPoint - feedback_value

        self.current_time = current_time if current_time is not None else time.time()
        delta_time = self.current_time - self.last_time
        delta_error = error - self.last_error

        if (delta_time >= self.sample_time):
            self.PTerm = self.Kp * error
            self.ITerm += error * delta_time

#             if (self.ITerm < -self.windup_guard):
#                 self.ITerm = -self.windup_guard
#             elif (self.ITerm > self.windup_guard):
#                 self.ITerm = self.windup_guard

            self.DTerm = 0.0
            if delta_time > 0:
                self.DTerm = delta_error / delta_time

            # Remember last time and last error for next calculation
            self.last_time = self.current_time
            self.last_error = error

            self.output = self.PTerm + (self.Ki * self.ITerm) + (self.Kd * self.DTerm)
        
        return self.output

    def setKs(self, proportional_gain, integral_gain, derivative_gain):
        """Determines how aggressively the PID reacts to the current error with setting Proportional Gain, Integral Gain and Derivative Gain"""
        self.Kp = proportional_gain
        self.Ki = integral_gain
        self.Kd = derivative_gain        

    def setWindup(self, windup):
        """Integral windup, also known as integrator windup or reset windup,
        refers to the situation in a PID feedback controller where
        a large change in setpoint occurs (say a positive change)
        and the integral terms accumulates a significant error
        during the rise (windup), thus overshooting and continuing
        to increase as this accumulated error is unwound
        (offset by errors in the other direction).
        The specific problem is the excess overshooting.
        """
        self.windup_guard = windup

    def setSampleTime(self, sample_time):
        """PID that should be updated at a regular interval.
        Based on a pre-determined sampe time, the PID decides if it should compute or return immediately.
        """
        self.sample_time = sample_time


In [ ]:
class Actor(object):
    """
        PPO Actor Neural Net
    """

    def __init__(self, state_dim, action_dim, pid, pid_parameters, action_bound, learning_rate, ratio_clipping):

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.pid_parameters = pid_parameters
        self.action_bound = action_bound
        self.learning_rate = learning_rate
        self.ratio_clipping = ratio_clipping

        # set min and max of standard deviation
        self.std_bound = [1e-2, 1.0]

        # create actor neural net
        self.model, self.theta = self.build_network()
        
        # pid
        self.pid = pid

        """
        tf 2.0
        """

        # loss and optimizer
        self.actor_optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate)

    def build_network(self):
        state_input = Input((self.state_dim,))
        h1 = Dense(64, activation='relu')(state_input)
        h2 = Dense(32, activation='relu')(h1)
        h3 = Dense(16, activation='relu')(h2)
        
        # pid_parameters : 3
        mu = Dense(self.pid_parameters, activation='tanh')(h3)
        std = Dense(self.pid_parameters, activation='softplus')(h3)
        
        model = Model(state_input, [mu, std])
        return model, model.trainable_weights

    def train(self, states, actions, advantages, log_old_policy_pdf):
        with tf.GradientTape() as g:
            mu, std = self.model(states)
            log_policy_pdf = self.log_pdf(mu, std , actions)

            # ratio of two policy & target function surrogate
            ratio = tf.exp(log_policy_pdf - log_old_policy_pdf)
            clipped_ratio = tf.clip_by_value(ratio, 1.0-self.ratio_clipping, 1.0 + self.ratio_clipping)
            surrogate = -tf.minimum(ratio * advantages, clipped_ratio * advantages)
            loss = tf.reduce_mean(surrogate)
        dj_dtheta = g.gradient(loss, self.theta)
        grads = zip(dj_dtheta, self.theta)
        self.actor_optimizer.apply_gradients(grads)

    # log_policy pdf
    def log_pdf(self, mu, std, action):
        std = tf.clip_by_value(std, self.std_bound[0], self.std_bound[1])
        var = std**2
        log_policy_pdf = -0.5 * (action - mu) ** 2 / var - 0.5 * tf.math.log(var * 2 * np.pi)
        return tf.reduce_sum(log_policy_pdf, 1, keepdims=True)

    # get agent's action
    def get_policy_action(self, state):
        mu, std = self.model(np.reshape(state, [1, self.state_dim]))
        mu, std = mu[0], std[0]
        std = tf.clip_by_value(std, self.std_bound[0], self.std_bound[1])
        
        p,i,d = np.random.normal(mu, std)
        action = self.get_action_from_pid(p, i, d, state)
        
        return mu, std, action
    
    # get action from pid
    def get_action_from_pid(self, p, i, d, state):
        self.pid.setKs(p,i,d)
        val = self.pid.update(np.array(state[:1]))
        action = val[1]/(val[0] + 0.000001)
        return [action]

    # calculate mean
    def predict(self, state):
        mu, _ = self.model(np.reshape(state, [1, self.state_dim]))
        action = self.get_action_from_pid(*mu, state)
        return action

    # save Actor parameter
    def save_weights(self, path):
        self.model.save_weights(path)

    # load Actor parameter
    def load_weights(self, path):
        self.model.load_weights(path+'pendulum_actor.h5')

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam

"""
    TF2.2 functional API
"""

class Critic(object):

    """
        PPO Critic Neural Net
    """

    def __init__(self, state_dim, action_dim, learning_rate):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.learning_rate = learning_rate

        # create critic neural net
        self.model = self.build_network()

        # set training method
        self.model.compile(optimizer=Adam(self.learning_rate), loss='mse')

    def build_network(self):
        state_input = Input((self.state_dim,))
        h1 = Dense(64, activation='relu')(state_input)
        h2 = Dense(32, activation='relu')(h1)
        h3 = Dense(16, activation='relu')(h2)
        v_output = Dense(1, activation='linear')(h3)

        model = Model(state_input, v_output)
        return model

    # update neural net with batch data
    def train_on_batch(self, states, td_targets):
        return self.model.train_on_batch(states, td_targets)

    # save Critic parameter
    def save_weights(self, path):
        self.model.save_weights(path)

    # load Critic parameter
    def load_weights(self, path):
        self.model.load_weights(path+'pendulum_critic.h5')


In [ ]:
class PPOagent(object):

    def __init__(self, env):

        # hyperparameter
        self.GAMMA = 0.95
        self.GAE_LAMBDA = 0.9
        self.BATCH_SIZE = 64
        self.ACTOR_LEARNING_RATE = 0.0001
        self.CRITIC_LEARNING_RATE = 0.001
        self.RATIO_CLIPPING = 0.2
        self.EPOCHS = 10
        self.t_MAX = 4

        # environment
        self.env = env
        # state dimension
        self.state_dim = env.observation_space.shape[0]
        # action dimension
        self.action_dim = env.action_space.shape[0]
        # max action size
        self.action_bound = env.action_space.high[0]
        
        # PID
        self.pid = PID()
        self.pid_parameters = 3

        # create Actor and Critic neural nets
        self.actor = Actor(self.state_dim, self.action_dim, self.pid, self.pid_parameters, 
                           self.action_bound, self.ACTOR_LEARNING_RATE, self.RATIO_CLIPPING)
        self.critic = Critic(self.state_dim, self.action_dim, self.CRITIC_LEARNING_RATE)

        # total reward of a episode
        self.save_epi_reward = []

    # calculate GAE and TD targets
    def gae_target(self, rewards, v_values, next_v_value, done):
        n_stop_targets = np.zeros_like(rewards)
        gae = np.zeros_like(rewards)
        gae_cumulative = 0
        forward_val = 0

        if not done:
            forward_val = next_v_value

        for k in reversed(range(0, len(rewards))):
            delta = rewards[k] + self.GAMMA * forward_val - v_values[k]
            gae_cumulative = self.GAMMA * self.GAE_LAMBDA * gae_cumulative + delta
            gae[k] = gae_cumulative
            forward_val = v_values[k]
            n_stop_targets[k] = gae[k] + v_values[k]
        return gae, n_stop_targets

    # train agent
    def train(self, max_episode_num):

        # repeat for each episode
        for ep in range(int(max_episode_num)):

            # init states, actions, reward
            states, actions, rewards = [], [], []
            # init log old policy pdf
            log_old_policy_pdfs = []
            # init episode
            time, episode_reward, done = 0, 0, False
            # reset env and observe initial state
            state = self.env.reset()

            while not done:

                # visualize env
                self.env.render()

                # get action
                mu_old, std_old, action = self.actor.get_policy_action(state)

                # bound action range
                action = np.clip(action, -self.action_bound, self.action_bound)

                # calculate log old policy pdf
                var_old = std_old**2
                log_old_policy_pdf = -0.5 * (action - mu_old)**2 / var_old - 0.5 * np.log(var_old * 2 * np.pi)
                log_old_policy_pdf = np.sum(log_old_policy_pdf)

                # observe next state, reward
                next_state, reward, done, _ = self.env.step(action)

                # save to batch
                states.append(state)
                actions.append(action)
                rewards.append((reward + 8) / 8)  # modify reward range
                log_old_policy_pdfs.append(log_old_policy_pdf)

                # state update
                state = next_state
                episode_reward += reward
                time += 1

                if len(states) == self.t_MAX or done:

                    # get data from batch
                    states = np.array(states)
                    actions = np.array(actions)
                    rewards = np.array(rewards)
                    log_old_policy_pdfs = np.array(log_old_policy_pdfs)


                    # calculate n-step TD target and advantage
                    next_state = np.reshape(next_state, [1, self.state_dim])
                    next_v_value = self.critic.model(next_state)
                    v_values = self.critic.model(states)
                    gaes, y_i = self.gae_target(rewards, v_values, next_v_value, done)

                    for _ in range(self.EPOCHS):
                        # train actor network
                        self.actor.train(states, actions, gaes, log_old_policy_pdfs)
                        # train critic network
                        self.critic.train_on_batch(states, y_i)

                    # clear batch
                    states, actions, rewards = [], [], []
                    log_old_policy_pdfs = []

            print("Epi: ", ep+1, "Time: ", time, "Reward: ", episode_reward, "PID : ", self.actor.pid.Kp,self.actor.pid.Ki,self.actor.pid.Kd)
            self.save_epi_reward.append(episode_reward)

            if ep % 10 == 0:
                self.actor.save_weights("./save_weights/pendulum_actor.h5")
                self.critic.save_weights("./save_weights/pendulum_actor.h5")

        np.savetxt('.save_weights/pendulum_epi_reward.txt', self.save_epi_reward)
        print(self.save_epi_reward)

    # graph episodes and rewards
    def plot_result(self):
        plt.plot(self.save_epi_reward)
        plt.show()

In [ ]:
def main():
    max_episode_num = 1000
    env_name = 'Pendulum-v0'
    env = gym.make(env_name)
    agent = PPOagent(env)
    
    # train
    agent.train(max_episode_num)
    
    # visualization
    agent.plot_result()
    
    env.close()

if __name__ =='__main__':
    main()

In [ ]:
# def main():
#     env_name = "Pendulum-v0"
#     env = gym.make(env_name)
#     agent = PPOagent(env_name)

#     agent.actor.load_weights('./save_weights/')
#     agent.critic.load_weights('./save_weights/')

#     time = 0

#     state = env.reset()

#     while True:
#         env.render()
#         action = agent.global_actor.predict(state)
#         state, reward, done, _ = env.step(action)
#         time += 1

#         print("Time: {}, Reward: {}".format(time, reward))

#         if done:
#             break

#     env.close()


# if __name__ == "__main__":
#     main()